## Problem 48 - Self Powers

<p>The series, $1^1 + 2^2 + 3^3 + \cdots + 10^{10} = 10405071317$.</p>
<p>Find the last ten digits of the series, $1^1 + 2^2 + 3^3 + \cdots + 1000^{1000}$.</p>


In [1]:
%%time

s = 0
for x in range(1, 1001, 1):
    s+= x**x

print(f"The answer is : {str(s)[-10:]}")

The answer is : 9110846700
CPU times: user 5.11 ms, sys: 494 µs, total: 5.6 ms
Wall time: 5.6 ms


## Problem 49 - Prime Permutations

<p>The arithmetic sequence, $1487, 4817, 8147$, in which each of the terms increases by $3330$, is unusual in two ways: (i) each of the three terms are prime, and, (ii) each of the $4$-digit numbers are permutations of one another.</p>
<p>There are no arithmetic sequences made up of three $1$-, $2$-, or $3$-digit primes, exhibiting this property, but there is one other $4$-digit increasing sequence.</p>
<p>What $12$-digit number do you form by concatenating the three terms in this sequence?</p>


In [2]:
from sympy import isprime
import numpy as np
from scipy.spatial.distance import cdist

#find four-digit-primes
fdps = np.array([x for x in range(1000, 10000,1) if isprime(x)])

In [3]:
def check_gap(fdp: int, gap: int):
    """Checks if for a four digit prime and a given gap, 
    the numbers found by adding or subtracting the gap are prime
    and also comprised of the same set of digits"""
    if set(str(fdp)) != set(str(fdp+gap)):
        return False
    
    if set(str(fdp)) != set(str(fdp-gap)):
        return False
    if not isprime(fdp+gap) or not isprime(fdp-gap):
        return False
    return True
        

In [4]:
#for every four digit prime
for fdp in fdps:
    # find all symmetrical gaps by looking for doubles in the sets of differences
    diffs, counts = np.unique(np.abs(fdps - fdp), return_counts=True)
    idxs = np.argwhere(counts == 2).flatten()
    gaps_to_check = diffs[idxs]
    #four every gap, check if the plus or minus number are prime and the same set, of so, log the instance
    for gap in gaps_to_check:
        if check_gap(fdp, gap):
            print(f"{fdp-gap}, {fdp}, {fdp+gap} with a gap of {gap}")

1487, 4817, 8147 with a gap of 3330
2969, 6299, 9629 with a gap of 3330


Interesting that the gap here is also 3330! I did not glean that from the excercise. Knowing that would have made the solution much easier I think. 

## Problem 50 - Consecutive Prime Sum

<p>The prime $41$, can be written as the sum of six consecutive primes:</p>
$$41 = 2 + 3 + 5 + 7 + 11 + 13.$$
<p>This is the longest sum of consecutive primes that adds to a prime below one-hundred.</p>
<p>The longest sum of consecutive primes below one-thousand that adds to a prime, contains $21$ terms, and is equal to $953$.</p>
<p>Which prime, below one-million, can be written as the sum of the most consecutive primes?</p>

-----
Note: The last sentence suggests that the length of the sequence we are looking for only ever produces one prime, which simplifies our search a little bit. We now only have to look for sliding windows where the rolling sum only produces one prime result:

In [90]:
%%time

#I`ll start with gathering all primes up to a million, even though we likely only need a few dozen at most
primes_to_million = [x for x in range(1000000) if isprime(x)]

#keep track of the largest sequence that produces one prime result
largest_seq = 0
#keep track ot the start index of where summing the sequence produces a prime result
start_index = 0

#for every window starting from 21:
for win in range(21, len(primes_to_million)):
    #take the sum of the windowlength over the entire array
    sums = np.sum(np.lib.stride_tricks.sliding_window_view(primes_to_million, win), axis = 1)
    #if none of the sums are under one million we have left the search space and can terminate the loop
    if np.min(sums) > 1e6:
        print(f"No sums under 1e6 left at window {win}, Stopping the search")
        break
    #check which sums are primes under one million
    ip = [isprime(x) and x<1e6 for x in sums]
    #if there is exactly one, we update the largest sequence and the starting index until we are done searching
    if sum(ip) == 1:
        largest_seq = win
        start_index = np.where(ip)[0][0]
#eventually we find the solution
print(f"The largest consecutive sequence of primes that sum to a prime under one million is  {largest_seq} long.\nIt starts at {primes_to_million[start_index]}. \nThe sum is {np.sum(primes_to_million[start_index:start_index+largest_seq])}")
  

No sums under 1e6 left at window 547, Stopping the search
The largest consecutive sequence of primes that sum to a prime under one million is  543 long and starts at 7. The sum is 997651
CPU times: user 54.3 s, sys: 20 ms, total: 54.4 s
Wall time: 54.3 s
